<a href="https://colab.research.google.com/github/cccaaannn/deep_learning_colab/blob/master/keras/cnn/testing_builtin_keras_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**check the gpu**

In [ ]:
!nvidia-smi

**install additional libs**

In [ ]:
!pip install imagepreprocessing

**imports**

In [ ]:
# keras and tf
import tensorflow as tf
import keras

# models
from keras.models import Sequential
from keras.models import Model

# backend
from keras import optimizers, metrics, models
import keras.backend as K

# layers
from keras.layers import Input, Flatten, Dense, Dropout, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization

# optimizers
from keras.optimizers import SGD

# training
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

# save
import h5py

# keras aplications
from keras.applications import DenseNet201, DenseNet169, InceptionResNetV2, ResNet152V2, InceptionV3, DenseNet121, Xception, MobileNet, VGG19


# other libs
import math
import pickle
import os

# imagepreprocessing
from  imagepreprocessing.keras_functions import create_history_graph_keras

**import keras aplications**

In [ ]:
# keras aplications
from tensorflow.keras.applications import (DenseNet121,
 DenseNet169,
 DenseNet201,
 EfficientNetB0,
 EfficientNetB1,
 EfficientNetB2,
 EfficientNetB3,
 EfficientNetB4,
 EfficientNetB5,
 EfficientNetB6,
 EfficientNetB7,
 InceptionResNetV2,
 InceptionV3,
 MobileNet,
 MobileNetV2,
 NASNetLarge,
 NASNetMobile,
 ResNet101,
 ResNet101V2,
 ResNet152,
 ResNet152V2,
 ResNet50,
 ResNet50V2,
 VGG16,
 VGG19)

**connect to the drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**set options**

In [ ]:
image_size = 224
batch_size = 32
nr_classes = 100
nr_ephocs = 30
train_path = "/content/drive/My Drive/Data/train"
test_path = "/content/drive/My Drive/Data/test"

# model_path = "/content/drive/My Drive/densenet201_100_class.h5"

**generate data**

In [ ]:
train_datagen = ImageDataGenerator(
        # rotation -40 40
        rotation_range=40,

        # shift images and fill pixels with nearest
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',

        horizontal_flip=True,
        zoom_range=0.3,

        shear_range=0.2,

        brightness_range=[0.1, 0.5],
        channel_shift_range=0.5,

        # normalization 
        featurewise_center=True,
        featurewise_std_normalization=True,
        # validation_split=0.2,
        )
valid_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)


train = train_datagen.flow_from_directory(train_path, shuffle=True, class_mode='categorical', target_size=(image_size,image_size), batch_size=batch_size)
valid = valid_datagen.flow_from_directory(test_path , shuffle=True, class_mode='categorical', target_size=(image_size,image_size), batch_size=batch_size)

**top k accuracy**

In [ ]:
import functools
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

top5_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=5)
top5_acc.__name__ = 'top5_acc'

top10_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=10)
top10_acc.__name__ = 'top10_acc'


**select model**

In [ ]:
base_model=DenseNet201(include_top=False)

In [ ]:
base_model=DenseNet169(include_top=False)

In [ ]:
base_model=DenseNet121(include_top=False)

In [ ]:
base_model=ResNet152V2(include_top=False)

In [ ]:
base_model=ResNet101V2(include_top=False)

In [ ]:
base_model=Xception(include_top=False)

In [ ]:
base_model=MobileNetV2(include_top=False)

In [ ]:
base_model=VGG19(include_top=False)

In [ ]:
base_model=VGG16(include_top=False)

In [ ]:
base_model=NASNetLarge(include_top=False)

In [ ]:
base_model=InceptionV3(include_top=False)

In [ ]:
base_model=InceptionResNetV2(include_top=False)

**add last layer**

In [ ]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
preds=Dense(nr_classes,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)

**compile model**

In [ ]:
opt = SGD(lr=0.001, momentum=0.9)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

In [ ]:
opt = SGD(lr=0.001, momentum=0.9)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy', top3_acc, top5_acc, top10_acc])

In [ ]:
opt = SGD(lr=0.001, nesterov=True)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy', top3_acc, top5_acc, top10_acc])

In [ ]:
opt = Adam(lr=0.001)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy', top3_acc, top5_acc, top10_acc])

In [ ]:
opt = RMSprop()
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy', top3_acc, top5_acc, top10_acc])

In [ ]:
opt = Adagrad()
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy', top3_acc, top5_acc, top10_acc])

**train**

In [ ]:
history = model.fit_generator(train, epochs=nr_ephocs, validation_data=valid)

**train with  checkpointer**

In [ ]:
checkpointer = ModelCheckpoint(filepath= model_path, monitor='val_loss', save_best_only=True)
history = model.fit_generator(train, epochs=nr_ephocs, validation_data=valid, callbacks=[checkpointer])

**create training history graph**

In [ ]:
create_history_graph_keras(history)
create_history_graph_keras(history, separate_train_val_plots=False)